# BicycleGAN

## 1. Import Libs

In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader

from dataloader import Edges2handbags
import util
import os
import argparse

from model import *

CUDA = torch.cuda.is_available()

## 2. Settings

In [2]:
parser = argparse.ArgumentParser([])

parser.add_argument('--root', required=False, default='./dataset/edges2shoes/edges2shoes')
parser.add_argument('--result_dir', default='result/edges2shoes')
parser.add_argument('--weight_dir', default='weight/edges2shoes')
parser.add_argument('--load_weight', type=bool, default=False)
parser.add_argument('--batch_size', type=int, default=2)
parser.add_argument('--test_size', type=int, default=20)
parser.add_argument('--test_img_num', type=int, default=5)
parser.add_argument('--img_size', type=int, default=128)
parser.add_argument('--num_epoch', type=int, default=100)
parser.add_argument('--save_every', type=int, default=1000)
parser.add_argument('--lr', type=float, default=0.0002)
parser.add_argument('--beta_1', type=float, default=0.5)
parser.add_argument('--beta_2', type=float, default=0.999)
parser.add_argument('--lambda_kl', type=float, default=0.01)
parser.add_argument('--lambda_img', type=int, default=10)
parser.add_argument('--lambda_z', type=float, default=0.5)
parser.add_argument('--z_dim', type=int, default=8)

params = parser.parse_args([])
print(params)

Namespace(batch_size=2, beta_1=0.5, beta_2=0.999, img_size=128, lambda_img=10, lambda_kl=0.01, lambda_z=0.5, load_weight=False, lr=0.0002, num_epoch=100, result_dir='result/edges2shoes', root='./dataset/edges2shoes/edges2shoes', save_every=1000, test_img_num=5, test_size=20, weight_dir='weight/edges2shoes', z_dim=8)


## 3. Utils

In [3]:
def mse_loss(score, target=1):
    dtype = type(score)
    
    if target == 1:
        label = util.var(torch.ones(score.size()), requires_grad=False)
    elif target == 0:
        label = util.var(torch.zeros(score.size()), requires_grad=False)
    
    criterion = nn.MSELoss().cuda()
    loss = criterion(score, label)
    
    return loss

def L1_loss(pred, target):
    return torch.mean(torch.abs(pred - target))

def lr_decay_rule(epoch, start_decay=100, lr_decay=100):
    decay_rate = 1.0 - (max(0, epoch - start_decay) / float(lr_decay))
    return decay_rate


def save_weight(epoch=None):
    if epoch is None:
        d_cVAE_name = 'D_cVAE.pkl'
        d_cLR_name = 'D_cLR.pkl'
        g_name = 'G.pkl'
        e_name = 'E.pkl'
    else:
        d_cVAE_name = '{epochs}-{name}'.format(epochs=str(epoch), name='D_cVAE.pkl')
        d_cLR_name = '{epochs}-{name}'.format(epochs=str(epoch), name='D_cLR.pkl')
        g_name = '{epochs}-{name}'.format(epochs=str(epoch), name='G.pkl')
        e_name = '{epochs}-{name}'.format(epochs=str(epoch), name='E.pkl')
            
    torch.save(D_cVAE.state_dict(), os.path.join(self.weight_dir, d_cVAE_name))
    torch.save(D_cVAE.state_dict(), os.path.join(self.weight_dir, d_cLR_name))
    torch.save(G.state_dict(), os.path.join(self.weight_dir, g_name))
    torch.save(E.state_dict(), os.path.join(self.weight_dir, e_name))

## 4. Load Dataset
### 4.1 Preprocessing

In [4]:
transform = transforms.Compose([
    transforms.Scale((params.img_size, params.img_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))])

### 4.2 DataLoader

In [5]:
train_dataset = Edges2handbags(params.root, transform=transform, mode='train')
test_dataset = Edges2handbags(params.root, transform=transform, mode='val')

train_data_loader = DataLoader(train_dataset, batch_size=params.batch_size, shuffle=True, num_workers=4, drop_last=True)
test_data_loader = DataLoader(test_dataset, batch_size=params.batch_size, shuffle=True, num_workers=4, drop_last=True)

train_sample = train_dataset.__getitem__(11)
print(train_sample)
print(len(train_sample))

(
( 0 ,.,.) = 
  1.0000  1.0000  1.0000  ...   1.0000  1.0000  1.0000
  1.0000  1.0000  1.0000  ...   1.0000  1.0000  1.0000
  1.0000  1.0000  1.0000  ...   1.0000  1.0000  1.0000
           ...             ⋱             ...          
  1.0000  1.0000  1.0000  ...   1.0000  1.0000  1.0000
  1.0000  1.0000  1.0000  ...   1.0000  1.0000  1.0000
  1.0000  1.0000  1.0000  ...   1.0000  1.0000  1.0000

( 1 ,.,.) = 
  1.0000  1.0000  1.0000  ...   1.0000  1.0000  1.0000
  1.0000  1.0000  1.0000  ...   1.0000  1.0000  1.0000
  1.0000  1.0000  1.0000  ...   1.0000  1.0000  1.0000
           ...             ⋱             ...          
  1.0000  1.0000  1.0000  ...   1.0000  1.0000  1.0000
  1.0000  1.0000  1.0000  ...   1.0000  1.0000  1.0000
  1.0000  1.0000  1.0000  ...   1.0000  1.0000  1.0000

( 2 ,.,.) = 
  1.0000  1.0000  1.0000  ...   1.0000  1.0000  1.0000
  1.0000  1.0000  1.0000  ...   1.0000  1.0000  1.0000
  1.0000  1.0000  1.0000  ...   1.0000  1.0000  1.0000
           ...        

## 5. Build Models & Optimizers & Loss Function

### 5.1 Models

In [6]:
D_cVAE = Discriminator().cuda()
D_cLR = Discriminator().cuda()
G = Generator(z_dim=params.z_dim).cuda()
E = Encoder(z_dim=params.z_dim).cuda()

print(D_cVAE)
print(D_cLR)
print(G)
print(E)

Discriminator(
  (d_1): Sequential(
    (0): AvgPool2d(kernel_size=3, stride=2, padding=0, ceil_mode=False, count_include_pad=False)
    (1): ConvBlock(
      (conv_block): Sequential(
        (0): Conv2d(3, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
        (1): LeakyReLU(0.2, inplace)
      )
    )
    (2): ConvBlock(
      (conv_block): Sequential(
        (0): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
        (1): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
      )
    )
    (3): ConvBlock(
      (conv_block): Sequential(
        (0): Conv2d(64, 128, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1))
        (1): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
      )
    )
    (4): ConvBlock(
      (conv_block): Sequential(
        (0): Conv2d(128, 1, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1))
      )
    )
  )
  (d_2): Sequential(
    (0): ConvBlock(
      (conv_block): Sequential(
        (0): Conv2d(3, 64, kerne

### 5.2 Optimizers

In [7]:
optim_D_cVAE = optim.Adam(D_cVAE.parameters(), lr=params.lr, betas=(params.beta_1, params.beta_2))
optim_D_cLR = optim.Adam(D_cLR.parameters(), lr=params.lr, betas=(params.beta_1, params.beta_2))
optim_G = optim.Adam(G.parameters(), lr=params.lr, betas=(params.beta_1, params.beta_2))
optim_E = optim.Adam(E.parameters(), lr=params.lr, betas=(params.beta_1, params.beta_2))

In [8]:
# 20 x 5 x 8
fixed_z = util.var(torch.randn(params.test_size, params.test_img_num, params.z_dim)).cuda()

## 6. Load Pretrained Models

In [9]:
if params.load_weight is True:
    D_cVAE.load_state_dict(torch.load(os.path.join(params.weight_dir, 'D_cVAE.pkl')))
    D_cLR.load_state_dict(torch.load(os.path.join(params.weight_dir, 'D_cLR.pkl')))
    G.load_state_dict(torch.load(os.path.join(params.weight_dir, 'G.pkl')))
    E.load_state_dict(torch.load(os.path.join(params.weight_dir, 'E.pkl')))

In [ ]:
D_cVAE.train()
D_cLR.train()
G.train()
E.train()

Encoder(
  (conv): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (res_blocks): Sequential(
    (0): ResBlock(
      (conv): Sequential(
        (0): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
        (1): LeakyReLU(0.2, inplace)
        (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (3): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
        (4): LeakyReLU(0.2, inplace)
        (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (6): AvgPool2d(kernel_size=2, stride=2, padding=0, ceil_mode=False, count_include_pad=True)
      )
      (short_cut): Sequential(
        (0): AvgPool2d(kernel_size=2, stride=2, padding=0, ceil_mode=False, count_include_pad=True)
        (1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1))
      )
    )
    (1): ResBlock(
      (conv): Sequential(
        (0): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
        (1): LeakyReLU(0.2, inplace)


In [ ]:
D_A_avg_losses = []
G_GAN_avg_losses = []
KL_div_avg_losses = []
img_recon_avg_losses = []
G_alone_avg_losses = []

for epoch in range(1, params.num_epoch + 1):
    D_A_losses = []
    G_GAN_losses = []
    KL_div_losses = []
    img_recon_losses = []
    G_alone_losses = []
    for iters, (img, ground_truth) in enumerate(train_data_loader): # 1
        # img : (2, 3, 128, 128) of domain A / ground_truth : (2, 3, 128, 128) of domain B
        img, ground_truth = util.var(img), util.var(ground_truth)
        
        # Seperate data for cVAE_GAN and cLR_GAN
        cVAE_data = {'img' : img[0].unsqueeze(dim=0), 'ground_truth' : ground_truth[0].unsqueeze(dim=0)}
        cLR_data = {'img' : img[1].unsqueeze(dim=0), 'ground_truth' : ground_truth[1].unsqueeze(dim=0)}
        
        ''' ----------------------------- 1. Train D ----------------------------- '''
        #############   Step 1. D loss in cVAE-GAN #############
        
        # Encoded latent vector
        mu, log_variance = E(cVAE_data['ground_truth'])
        std = torch.exp(log_variance / 2)
        random_z = util.var(torch.randn(1, params.z_dim))
        encoded_z = (random_z * std) + mu
        
        # Generate fake image
        fake_img_cVAE = G(cVAE_data['img'], encoded_z)
        
        # Get scores and loss
        real_d_cVAE_1, real_d_cVAE_2 = D_cVAE(cVAE_data['ground_truth'])
        fake_d_cVAE_1, fake_d_cVAE_2 = D_cVAE(fake_img_cVAE)
                
        # mse_loss for LSGAN
        D_loss_cVAE_1 = mse_loss(real_d_cVAE_1, 1) + mse_loss(fake_d_cVAE_1, 0)
        D_loss_cVAE_2 = mse_loss(real_d_cVAE_2, 1) + mse_loss(fake_d_cVAE_2, 0)
                
        #############   Step 2. D loss in cLR-GAN   #############

        # Random latent vector
        random_z = util.var(torch.randn(1, params.z_dim))

        # Generate fake image
        fake_img_cLR = G(cLR_data['img'], random_z)

        # Get scores and loss
        real_d_cLR_1, real_d_cLR_2 = D_cLR(cLR_data['ground_truth'])
        fake_d_cLR_1, fake_d_cLR_2 = D_cLR(fake_img_cLR)
        
        D_loss_cLR_1 = mse_loss(real_d_cLR_1, 1) + mse_loss(fake_d_cLR_1, 0)
        D_loss_cLR_2 = mse_loss(real_d_cLR_2, 1) + mse_loss(fake_d_cLR_2, 0)
        
        D_loss = D_loss_cVAE_1 + D_loss_cLR_1 + D_loss_cVAE_2 + D_loss_cLR_2
        
        # Update
        optim_D_cVAE.zero_grad()
        optim_D_cLR.zero_grad()
        optim_G.zero_grad()
        optim_E.zero_grad()
        D_loss.backward()
        optim_D_cVAE.step()
        optim_D_cLR.step()
        
        ''' ----------------------------- 2. Train G & E ----------------------------- '''
        ############# Step 1. GAN loss to fool discriminator (cVAE_GAN and cLR_GAN) #############
        
        # Encoded latent vector
        mu, log_variance = E(cVAE_data['ground_truth'])
        std = torch.exp(log_variance / 2)
        random_z = util.var(torch.randn(1, params.z_dim))
        encoded_z = (random_z * std) + mu
        
        # Generate fake image and get adversarial loss
        fake_img_cVAE = G(cVAE_data['img'], encoded_z)
        fake_d_cVAE_1, fake_d_cVAE_2 = D_cVAE(fake_img_cVAE)
        
        GAN_loss_cVAE_1 = mse_loss(fake_d_cVAE_1, 1)
        GAN_loss_cVAE_2 = mse_loss(fake_d_cVAE_2, 1)
        
        # Random latent vector
        random_z = util.var(torch.randn(1, params.z_dim))
        
        # Generate fake image and get adversarial loss
        fake_img_cLR = G(cLR_data['img'], random_z)
        fake_d_cLR_1, fake_d_cLR_2 = D_cLR(fake_img_cLR)
        
        GAN_loss_cLR_1 = mse_loss(fake_d_cLR_1, 1)
        GAN_loss_cLR_2 = mse_loss(fake_d_cLR_2, 1)
        
        G_GAN_loss = GAN_loss_cVAE_1 + GAN_loss_cVAE_2 + GAN_loss_cLR_1 + GAN_loss_cLR_2
        
        ############# Step 2. KL-divergence with N(0, 1) (cVAE-GAN) #############
                
        KL_div = params.lambda_kl * torch.sum(0.5 * (mu ** 2 + torch.exp(log_variance) - log_variance - 1))
        
        ############# Step 3. Reconstruction of ground truth image (|G(A, z) - B|) (cVAE-GAN) #############
        img_recon_loss = params.lambda_img * L1_loss(fake_img_cVAE, cVAE_data['ground_truth'])
        
        EG_loss = G_GAN_loss + KL_div + img_recon_loss
        optim_D_cVAE.zero_grad()
        optim_D_cLR.zero_grad()
        optim_G.zero_grad()
        optim_E.zero_grad()
        EG_loss.backward(retain_graph=True)
        optim_E.step()
        optim_G.step()
        
        ''' ----------------------------- 3. Train ONLY G ----------------------------- '''
        ############ Step 1. Reconstrution of random latent code (|E(G(A, z)) - z|) (cLR-GAN) ############
                
        # This step should update ONLY G.
        mu_, log_variance_ = E(fake_img_cLR)
        z_recon_loss = L1_loss(mu_, random_z)
        
        G_alone_loss = params.lambda_z * z_recon_loss
        
        optim_D_cVAE.zero_grad()
        optim_D_cLR.zero_grad()
        optim_G.zero_grad()
        optim_E.zero_grad()
        G_alone_loss.backward()
        optim_G.step()
                        
        # Print error and save intermediate result image and weight
        D_A_losses.append(D_loss.data[0])
        G_GAN_losses.append(G_GAN_loss.data[0])
        KL_div_losses.append(KL_div.data[0])
        img_recon_losses.append(img_recon_loss.data[0])
        G_alone_losses.append(G_alone_loss.data[0])
        
        if iters % params.save_every == 0:
            print('[Epoch : %d / Iters : %d/%d] => D_loss : %f / G_GAN_loss : %f / KL_div : %f / img_recon_loss : %f / z_recon_loss : %f'\
                  %(epoch, iters, len(train_data_loader),D_loss.data[0], G_GAN_loss.data[0], KL_div.data[0], img_recon_loss.data[0], G_alone_loss.data[0]))
            
            # Save intermediate result image
            if os.path.exists(params.result_dir) is False:
                os.makedirs(params.result_dir)
                
            result_img = util.make_img(test_data_loader, G, fixed_z, img_num=params.test_img_num, img_size=params.img_size)
                
            img_name = '{epoch}_{iters}.png'.format(epoch=epoch, iters=iters)
            img_path = os.path.join(params.result_dir, img_name)
            
            torchvision.utils.save_image(result_img, img_path, nrow=params.test_img_num+1)
                
            # Save intermediate weight
            if os.path.exists(params.weight_dir) is False:
                os.makedirs(params.weight_dir)
                
            if epoch is None:
                d_cVAE_name = 'D_cVAE.pkl'
                d_cLR_name = 'D_cLR.pkl'
                g_name = 'G.pkl'
                e_name = 'E.pkl'
            else:
                d_cVAE_name = '{epochs}-{name}'.format(epochs=str(epoch), name='D_cVAE.pkl')
                d_cLR_name = '{epochs}-{name}'.format(epochs=str(epoch), name='D_cLR.pkl')
                g_name = '{epochs}-{name}'.format(epochs=str(epoch), name='G.pkl')
                e_name = '{epochs}-{name}'.format(epochs=str(epoch), name='E.pkl')
                
            torch.save(D_cVAE.state_dict(), os.path.join(params.weight_dir, d_cVAE_name))
            torch.save(D_cVAE.state_dict(), os.path.join(params.weight_dir, d_cLR_name))
            torch.save(G.state_dict(), os.path.join(params.weight_dir, g_name))
            torch.save(E.state_dict(), os.path.join(params.weight_dir, e_name))
    
    D_A_avg_loss = torch.mean(torch.FloatTensor(D_A_losses))
    G_GAN_avg_loss = torch.mean(torch.FloatTensor(G_GAN_losses))
    KL_div_avg_loss = torch.mean(torch.FloatTensor(KL_div_losses))
    img_recon_avg_loss = torch.mean(torch.FloatTensor(img_recon_losses))
    G_alone_avg_loss = torch.mean(torch.FloatTensor(G_alone_losses))
    
    D_A_avg_losses.append(D_A_avg_loss)
    G_GAN_avg_losses.append(G_GAN_avg_loss)
    KL_div_avg_losses.append(KL_div_avg_loss)
    img_recon_avg_losses.append(img_recon_avg_loss)
    G_alone_avg_losses.append(G_alone_avg_loss)
    
    # Save weight at the end of every epoch
    if epoch is None:
        d_cVAE_name = 'D_cVAE.pkl'
        d_cLR_name = 'D_cLR.pkl'
        g_name = 'G.pkl'
        e_name = 'E.pkl'
    else:
        d_cVAE_name = '{epochs}-{name}'.format(epochs=str(epoch), name='D_cVAE.pkl')
        d_cLR_name = '{epochs}-{name}'.format(epochs=str(epoch), name='D_cLR.pkl')
        g_name = '{epochs}-{name}'.format(epochs=str(epoch), name='G.pkl')
        e_name = '{epochs}-{name}'.format(epochs=str(epoch), name='E.pkl')
                
    torch.save(D_cVAE.state_dict(), os.path.join(params.weight_dir, d_cVAE_name))
    torch.save(D_cVAE.state_dict(), os.path.join(params.weight_dir, d_cLR_name))
    torch.save(G.state_dict(), os.path.join(params.weight_dir, g_name))
    torch.save(E.state_dict(), os.path.join(params.weight_dir, e_name))

[Epoch : 1 / Iters : 0/24912] => D_loss : 4.603251 / G_GAN_loss : 4.159481 / KL_div : 0.000277 / img_recon_loss : 7.558178 / z_recon_loss : 0.456559
[Epoch : 1 / Iters : 1000/24912] => D_loss : 1.981351 / G_GAN_loss : 1.223374 / KL_div : 0.047536 / img_recon_loss : 1.530019 / z_recon_loss : 0.390908
[Epoch : 1 / Iters : 2000/24912] => D_loss : 2.123360 / G_GAN_loss : 1.305619 / KL_div : 0.060278 / img_recon_loss : 1.102830 / z_recon_loss : 0.226525
[Epoch : 1 / Iters : 3000/24912] => D_loss : 1.948047 / G_GAN_loss : 0.880877 / KL_div : 0.031289 / img_recon_loss : 0.880946 / z_recon_loss : 0.450478
[Epoch : 1 / Iters : 4000/24912] => D_loss : 2.162253 / G_GAN_loss : 1.074162 / KL_div : 0.057745 / img_recon_loss : 1.648805 / z_recon_loss : 0.275282
[Epoch : 1 / Iters : 5000/24912] => D_loss : 1.982303 / G_GAN_loss : 1.473475 / KL_div : 0.024556 / img_recon_loss : 1.875346 / z_recon_loss : 0.263723
[Epoch : 1 / Iters : 6000/24912] => D_loss : 1.724704 / G_GAN_loss : 1.327337 / KL_div : 0.

[Epoch : 3 / Iters : 4000/24912] => D_loss : 1.841560 / G_GAN_loss : 1.297620 / KL_div : 0.065383 / img_recon_loss : 1.234728 / z_recon_loss : 0.459872
[Epoch : 3 / Iters : 5000/24912] => D_loss : 2.145462 / G_GAN_loss : 1.042770 / KL_div : 0.042922 / img_recon_loss : 0.833302 / z_recon_loss : 0.300104
[Epoch : 3 / Iters : 6000/24912] => D_loss : 1.687431 / G_GAN_loss : 1.360591 / KL_div : 0.091470 / img_recon_loss : 1.264833 / z_recon_loss : 0.267407
[Epoch : 3 / Iters : 7000/24912] => D_loss : 2.063912 / G_GAN_loss : 1.225348 / KL_div : 0.066107 / img_recon_loss : 1.143244 / z_recon_loss : 0.177391
[Epoch : 3 / Iters : 8000/24912] => D_loss : 2.292648 / G_GAN_loss : 0.851084 / KL_div : 0.063794 / img_recon_loss : 0.670752 / z_recon_loss : 0.406215
[Epoch : 3 / Iters : 9000/24912] => D_loss : 1.717583 / G_GAN_loss : 1.293704 / KL_div : 0.075155 / img_recon_loss : 1.278809 / z_recon_loss : 0.276675
[Epoch : 3 / Iters : 10000/24912] => D_loss : 1.803439 / G_GAN_loss : 1.289641 / KL_div 

[Epoch : 5 / Iters : 8000/24912] => D_loss : 2.112868 / G_GAN_loss : 0.823576 / KL_div : 0.085143 / img_recon_loss : 1.144694 / z_recon_loss : 0.508547
[Epoch : 5 / Iters : 9000/24912] => D_loss : 1.897226 / G_GAN_loss : 1.095377 / KL_div : 0.088128 / img_recon_loss : 0.597625 / z_recon_loss : 0.254169
[Epoch : 5 / Iters : 10000/24912] => D_loss : 1.531724 / G_GAN_loss : 1.110777 / KL_div : 0.076599 / img_recon_loss : 1.515151 / z_recon_loss : 0.380637
[Epoch : 5 / Iters : 11000/24912] => D_loss : 1.946244 / G_GAN_loss : 0.989890 / KL_div : 0.120858 / img_recon_loss : 0.729216 / z_recon_loss : 0.333066
[Epoch : 5 / Iters : 12000/24912] => D_loss : 2.033694 / G_GAN_loss : 1.194821 / KL_div : 0.090540 / img_recon_loss : 0.654018 / z_recon_loss : 0.310838
[Epoch : 5 / Iters : 13000/24912] => D_loss : 1.802564 / G_GAN_loss : 1.073138 / KL_div : 0.082131 / img_recon_loss : 0.930791 / z_recon_loss : 0.274080
[Epoch : 5 / Iters : 14000/24912] => D_loss : 2.238050 / G_GAN_loss : 1.080336 / KL_

[Epoch : 7 / Iters : 12000/24912] => D_loss : 1.832914 / G_GAN_loss : 0.911621 / KL_div : 0.099744 / img_recon_loss : 1.172057 / z_recon_loss : 0.376637
[Epoch : 7 / Iters : 13000/24912] => D_loss : 1.471245 / G_GAN_loss : 1.221214 / KL_div : 0.075535 / img_recon_loss : 1.033417 / z_recon_loss : 0.378407
[Epoch : 7 / Iters : 14000/24912] => D_loss : 2.113875 / G_GAN_loss : 1.109149 / KL_div : 0.066619 / img_recon_loss : 0.626220 / z_recon_loss : 0.297430
[Epoch : 7 / Iters : 15000/24912] => D_loss : 1.872684 / G_GAN_loss : 0.855463 / KL_div : 0.066350 / img_recon_loss : 0.701629 / z_recon_loss : 0.276716
[Epoch : 7 / Iters : 16000/24912] => D_loss : 2.105702 / G_GAN_loss : 1.065383 / KL_div : 0.063630 / img_recon_loss : 0.469000 / z_recon_loss : 0.502947
[Epoch : 7 / Iters : 17000/24912] => D_loss : 1.456427 / G_GAN_loss : 1.411275 / KL_div : 0.089081 / img_recon_loss : 2.564306 / z_recon_loss : 0.273868
[Epoch : 7 / Iters : 18000/24912] => D_loss : 1.786429 / G_GAN_loss : 1.030976 / K

[Epoch : 9 / Iters : 16000/24912] => D_loss : 1.543392 / G_GAN_loss : 1.991151 / KL_div : 0.124379 / img_recon_loss : 1.173128 / z_recon_loss : 0.390819
[Epoch : 9 / Iters : 17000/24912] => D_loss : 1.902954 / G_GAN_loss : 1.252480 / KL_div : 0.075324 / img_recon_loss : 0.699395 / z_recon_loss : 0.250629
[Epoch : 9 / Iters : 18000/24912] => D_loss : 1.944536 / G_GAN_loss : 1.125119 / KL_div : 0.076816 / img_recon_loss : 0.710088 / z_recon_loss : 0.329106
[Epoch : 9 / Iters : 19000/24912] => D_loss : 1.358475 / G_GAN_loss : 1.523847 / KL_div : 0.073403 / img_recon_loss : 0.627046 / z_recon_loss : 0.235112
[Epoch : 9 / Iters : 20000/24912] => D_loss : 1.983594 / G_GAN_loss : 0.956756 / KL_div : 0.045249 / img_recon_loss : 0.437470 / z_recon_loss : 0.430519
[Epoch : 9 / Iters : 21000/24912] => D_loss : 2.003389 / G_GAN_loss : 1.167075 / KL_div : 0.057453 / img_recon_loss : 0.460713 / z_recon_loss : 0.253933
[Epoch : 9 / Iters : 22000/24912] => D_loss : 1.639707 / G_GAN_loss : 1.054391 / K

[Epoch : 11 / Iters : 20000/24912] => D_loss : 2.212799 / G_GAN_loss : 0.955905 / KL_div : 0.062720 / img_recon_loss : 0.583279 / z_recon_loss : 0.390643
[Epoch : 11 / Iters : 21000/24912] => D_loss : 1.785781 / G_GAN_loss : 1.243750 / KL_div : 0.084390 / img_recon_loss : 0.758981 / z_recon_loss : 0.415946
[Epoch : 11 / Iters : 22000/24912] => D_loss : 1.755759 / G_GAN_loss : 1.249485 / KL_div : 0.088437 / img_recon_loss : 1.171426 / z_recon_loss : 0.240628
[Epoch : 11 / Iters : 23000/24912] => D_loss : 2.105589 / G_GAN_loss : 1.209806 / KL_div : 0.060779 / img_recon_loss : 0.419024 / z_recon_loss : 0.282072
[Epoch : 11 / Iters : 24000/24912] => D_loss : 1.746472 / G_GAN_loss : 1.439091 / KL_div : 0.092814 / img_recon_loss : 0.573856 / z_recon_loss : 0.351320
[Epoch : 12 / Iters : 0/24912] => D_loss : 1.874107 / G_GAN_loss : 1.102705 / KL_div : 0.073077 / img_recon_loss : 0.957539 / z_recon_loss : 0.281166
[Epoch : 12 / Iters : 1000/24912] => D_loss : 1.712808 / G_GAN_loss : 1.403944 /

[Epoch : 13 / Iters : 24000/24912] => D_loss : 2.130201 / G_GAN_loss : 0.922512 / KL_div : 0.094246 / img_recon_loss : 0.573079 / z_recon_loss : 0.282360
[Epoch : 14 / Iters : 0/24912] => D_loss : 1.663135 / G_GAN_loss : 1.334234 / KL_div : 0.108502 / img_recon_loss : 1.207019 / z_recon_loss : 0.255903
[Epoch : 14 / Iters : 1000/24912] => D_loss : 2.067245 / G_GAN_loss : 1.106416 / KL_div : 0.061687 / img_recon_loss : 0.411227 / z_recon_loss : 0.222534
[Epoch : 14 / Iters : 2000/24912] => D_loss : 1.813477 / G_GAN_loss : 1.084201 / KL_div : 0.123841 / img_recon_loss : 1.296911 / z_recon_loss : 0.269654
[Epoch : 14 / Iters : 3000/24912] => D_loss : 2.203998 / G_GAN_loss : 0.998572 / KL_div : 0.047542 / img_recon_loss : 0.342237 / z_recon_loss : 0.208416
[Epoch : 14 / Iters : 4000/24912] => D_loss : 1.897152 / G_GAN_loss : 1.132323 / KL_div : 0.079951 / img_recon_loss : 0.512777 / z_recon_loss : 0.245862
[Epoch : 14 / Iters : 5000/24912] => D_loss : 1.923073 / G_GAN_loss : 1.056539 / KL_

[Epoch : 16 / Iters : 3000/24912] => D_loss : 1.851317 / G_GAN_loss : 0.992207 / KL_div : 0.096333 / img_recon_loss : 0.564751 / z_recon_loss : 0.473993
[Epoch : 16 / Iters : 4000/24912] => D_loss : 1.986242 / G_GAN_loss : 0.985550 / KL_div : 0.076164 / img_recon_loss : 0.785625 / z_recon_loss : 0.171234
[Epoch : 16 / Iters : 5000/24912] => D_loss : 2.075979 / G_GAN_loss : 1.212994 / KL_div : 0.084929 / img_recon_loss : 0.896066 / z_recon_loss : 0.221146
[Epoch : 16 / Iters : 6000/24912] => D_loss : 1.771741 / G_GAN_loss : 1.216447 / KL_div : 0.080943 / img_recon_loss : 1.008126 / z_recon_loss : 0.300882
[Epoch : 16 / Iters : 7000/24912] => D_loss : 2.078640 / G_GAN_loss : 1.143723 / KL_div : 0.064087 / img_recon_loss : 0.649640 / z_recon_loss : 0.344369
[Epoch : 16 / Iters : 8000/24912] => D_loss : 2.084231 / G_GAN_loss : 1.589986 / KL_div : 0.085770 / img_recon_loss : 0.704288 / z_recon_loss : 0.352173
[Epoch : 16 / Iters : 9000/24912] => D_loss : 2.099638 / G_GAN_loss : 1.043879 / K

[Epoch : 18 / Iters : 8000/24912] => D_loss : 2.073735 / G_GAN_loss : 0.920242 / KL_div : 0.045156 / img_recon_loss : 0.336219 / z_recon_loss : 0.227927
[Epoch : 18 / Iters : 9000/24912] => D_loss : 1.822137 / G_GAN_loss : 1.342276 / KL_div : 0.053611 / img_recon_loss : 0.420848 / z_recon_loss : 0.327007
[Epoch : 18 / Iters : 10000/24912] => D_loss : 1.935140 / G_GAN_loss : 1.134310 / KL_div : 0.111852 / img_recon_loss : 0.772468 / z_recon_loss : 0.365200
[Epoch : 18 / Iters : 11000/24912] => D_loss : 1.961246 / G_GAN_loss : 1.115086 / KL_div : 0.073031 / img_recon_loss : 0.685716 / z_recon_loss : 0.317659
[Epoch : 18 / Iters : 12000/24912] => D_loss : 2.083831 / G_GAN_loss : 1.024708 / KL_div : 0.079081 / img_recon_loss : 0.516661 / z_recon_loss : 0.564901
[Epoch : 18 / Iters : 13000/24912] => D_loss : 1.898313 / G_GAN_loss : 1.054459 / KL_div : 0.107894 / img_recon_loss : 0.704600 / z_recon_loss : 0.192709
[Epoch : 18 / Iters : 14000/24912] => D_loss : 1.946187 / G_GAN_loss : 1.26456

[Epoch : 20 / Iters : 12000/24912] => D_loss : 1.850833 / G_GAN_loss : 1.275510 / KL_div : 0.088489 / img_recon_loss : 0.998211 / z_recon_loss : 0.332531
[Epoch : 20 / Iters : 13000/24912] => D_loss : 1.970487 / G_GAN_loss : 1.215244 / KL_div : 0.092297 / img_recon_loss : 1.298357 / z_recon_loss : 0.256384
[Epoch : 20 / Iters : 14000/24912] => D_loss : 1.362746 / G_GAN_loss : 1.037199 / KL_div : 0.096765 / img_recon_loss : 1.031890 / z_recon_loss : 0.226729
[Epoch : 20 / Iters : 15000/24912] => D_loss : 1.934073 / G_GAN_loss : 1.176131 / KL_div : 0.072267 / img_recon_loss : 0.694314 / z_recon_loss : 0.229572
[Epoch : 20 / Iters : 16000/24912] => D_loss : 1.914967 / G_GAN_loss : 0.951851 / KL_div : 0.128770 / img_recon_loss : 0.509386 / z_recon_loss : 0.299146
[Epoch : 20 / Iters : 17000/24912] => D_loss : 2.085118 / G_GAN_loss : 1.182284 / KL_div : 0.060245 / img_recon_loss : 0.563802 / z_recon_loss : 0.194118
[Epoch : 20 / Iters : 18000/24912] => D_loss : 1.542733 / G_GAN_loss : 1.007